<a href="https://colab.research.google.com/github/Srichandanak/sdp/blob/main/My_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pandas
import pandas as pd

In [4]:
import numpy as np
# Load and clean the data
bolly_df = pd.read_csv('/content/drive/MyDrive/bollywood_meta_2010-2019.csv')
bolly_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
bolly_df.fillna('True', inplace=True)  # Example: fill unnamed columns with 'True'
bolly_df.columns = [col if not col.startswith('Unnamed') else 'appropriate_name' for col in bolly_df.columns]
bolly_df.replace('\\N', np.nan, inplace=True)
bolly_df.dropna(inplace=True)
# Display cleaned DataFrame
print(bolly_df)

       imdb_id                      title             original_title  \
0    tt0073707                     Sholay                     Sholay   
1    tt0246825               Om Dar-B-Dar               Om Dar-B-Dar   
2    tt0403935             Action Jackson             Action Jackson   
3    tt0443331            Milenge Milenge            Milenge Milenge   
4    tt0839742  Bhopal: A Prayer for Rain  Bhopal: A Prayer for Rain   
..         ...                        ...                        ...   
921  tt9420648                       Bala                       Bala   
922  tt9472208              Battalion 609              Battalion 609   
924  tt9496212                   22 Yards                   22 Yards   
925  tt9558612           PM Narendra Modi           PM Narendra Modi   
928  tt9795264                  Risknamaa                  Risknamaa   

     is_adult year_of_release runtime                   genres  
0           0            1975     198  Action|Adventure|Comedy  
1    

In [5]:
# Add age of cinema
from datetime import datetime
year = datetime.now().year

bolly_df['year_of_release'] = pd.to_numeric(bolly_df['year_of_release'], errors='coerce')
bolly_df['ageofcinema'] = year - bolly_df['year_of_release']



In [6]:
# One-hot encode genres
bolly_df_gen = bolly_df['genres'].str.get_dummies(sep='|')
bolly_df = pd.concat([bolly_df, bolly_df_gen], axis=1)
bolly_df.drop('genres', inplace=True, axis=1)


In [7]:
age_groups = [17, 26, 35, 42, 50]

# Initialize all age group columns with False
for i, age in enumerate(age_groups):
    if i == 0:
        bolly_df[f'agegrp_{age}'] = bolly_df['ageofcinema'] <= age
    else:
        bolly_df[f'agegrp_{age}'] = (bolly_df['ageofcinema'] > age_groups[i-1]) & (bolly_df['ageofcinema'] <= age)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Define features and target variable, exclude 'title' from features
features = bolly_df[['ageofcinema', 'Action', 'Adventure', 'Animation', 'Biography',
                     'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
                     'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance',
                     'Sci-Fi', 'Sport', 'Thriller', 'War','agegrp_17','agegrp_26','agegrp_35','agegrp_42','agegrp_50']]
target = bolly_df[['agegrp_17','agegrp_26','agegrp_35','agegrp_42','agegrp_50']]



In [9]:
from sklearn.model_selection import train_test_split
# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)



In [10]:
# Initialize and train the MultiOutput Random Forest model
rf_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
rf_model.fit(X_train, y_train)


MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [11]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')


Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       175
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00       175
   macro avg       0.20      0.20      0.20       175
weighted avg       1.00      1.00      1.00       175
 samples avg       1.00      1.00      1.00       175



In [12]:
# Define function to get feature columns (excluding title)
def get_feature_columns():
    feature_columns = [
        'ageofcinema', 'Action', 'Adventure', 'Animation', 'Biography',
        'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
        'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance',
        'Sci-Fi', 'Sport', 'Thriller', 'War', 'agegrp_17', 'agegrp_26',
        'agegrp_35', 'agegrp_42', 'agegrp_50'
    ]
    return feature_columns


In [13]:
# Define function to create user input DataFrame
def create_user_df(age, genres, feature_columns):
    user_input = {col: [0] for col in feature_columns}
    user_input['ageofcinema'] = [age]
    for genre in genres:
        if genre in user_input:
            user_input[genre] = [1]
    # Calculate age groups
    age_groups = [17, 26, 35, 42, 50]
    for age_group in age_groups:
        user_input[f'agegrp_{age_group}'] = [age <= age_group]
    user_df = pd.DataFrame(user_input)
    return user_df


In [20]:
def recommend_movies(model, age, genres, original_df):
    feature_columns = get_feature_columns()
    user_df = create_user_df(age, genres, feature_columns)
    # Predict probabilities for each age group
    predictions = model.predict_proba(user_df)

    # Initialize variables to store recommendations
    recommended_movies = []
    for idx, age_group in enumerate(age_groups):
        # Filter original dataframe based on genres
        filtered_df = original_df.copy()
        for genre in genres:
            if genre in filtered_df:
                filtered_df = filtered_df[filtered_df[genre] == 1]
                 #Determine the age group based on the given age
    if age <= 17:
        age_group_col = 'agegrp_17'
    elif age <= 26:
        age_group_col = 'agegrp_26'
    elif age <= 35:
        age_group_col = 'agegrp_35'
    elif age <= 42:
        age_group_col = 'agegrp_42'
    else:
        age_group_col = 'agegrp_50'
    for i in range(5):  # Assuming you want to iterate up to 5 times
        filtered_df = filtered_df[filtered_df[age_group_col] == True]
        filtered_df = filtered_df.sort_values(by='ageofcinema', ascending=True)
        if not filtered_df.empty:
            recommended_movies.extend(filtered_df.iloc[:n, filtered_df.columns.get_loc('title')].tolist())
# Stop once we have enough recommendations
            if len(recommended_movies) >= n:
                recommended_movies = recommended_movies[:n]  # Limit to 'n' recommendations
                break
        else:
          for idx, age_group in enumerate(age_groups):
# Filter original dataframe based on genres
                filtered_df = original_df.copy()
                for genre in genres:
                     if genre in filtered_df:
                            filtered_df = filtered_df[filtered_df[genre] == 1]
                            filtered_df = filtered_df.sort_values(by='ageofcinema', ascending=True)
                            recommended_movies.extend(filtered_df.iloc[:n, filtered_df.columns.get_loc('title')].tolist())
                     if len(recommended_movies) >= n:
                           recommended_movies = recommended_movies[:n]  # Limit to 'n' recommendations
                           #recommended_movies.sort(by='ageofcinema',ascending=True)
                           break
    return recommended_movies



In [24]:
age = int(input())
preferred_genres = input().strip().split(",")
n = 100

print("\nRecommended Movies:")
recommended_movie_list = recommend_movies(rf_model, age, preferred_genres, bolly_df)
print(*recommended_movie_list,sep="\n")


25
Comedy

Recommended Movies:
Bala
Fraud Saiyaan
Bombairiya
Total Dhamaal
Housefull 4
Arjun Patiala
Judgementall Hai Kya
Jhootha Kahin Ka
The Zoya Factor
Dream Girl
Jabariya Jodi
De De Pyaar De
Made in China
Ujda Chaman
Motichoor Chaknachoor
Ishqeria
Veerey Ki Wedding
Hope Aur Hum
Simmba
Made in India
Phamous
Nanu Ki Jaanu
Rajma Chawal
5 Weddings
Hichki
Zero
Jack & Dil
Brother Superhit!
FryDay
Khajoor Pe Atke
Sonu Ke Titu Ki Sweety
Pataakha
High Jack
Kaalakaandi
Prince's Son
Mard Ko Dard Nahin Hota
Karwaan
Happy Phirr Bhag Jayegi
Stree
Dil Juunglee
Namaste England
Badhaai Ho
Mitron
Dhadak
Yamla Pagla Deewana Phir Se...
Love Per Square Foot
Jab Harry Met Sejal
Toilet - Ek Prem Katha
Noor
Hindi Medium
Jagga Jasoos
A Gentleman
Phillauri
Meri Pyaari Bindu
Twins 2
Bank Chor
Chef
Sweetiee Weds NRI
Patel Ki Punjabi Shaadi
Hind Ka Napak Ko Jawab
Tumhari Sulu
Bhanwarey
Golmaal Again
Bareilly Ki Barfi
Behen Hogi Teri
Long Live Brij Mohan
Fukrey Returns
Poster Boys
Coffee with D
Jattu Engineer
R